### ssh eddies ###

__data__

In [ ]:
import datetime, jdcal
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
ds = xr.open_dataset('/home/cz2397/data/ssh-eddies/tracks_AVISO_DT2014_daily_web.nc', decode_cf=False)

In [ ]:
df = ds.to_dataframe()

In [ ]:
df.head()

__time__

In [ ]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [ ]:
date_fix = df.j1.apply(jday_to_datetime)

In [ ]:
df.j1 = date_fix
df = df.rename(columns = {'j1': 'date'})

In [ ]:
df.head()

__longitude__

In [ ]:
# longitude range setting
# from -180 to 180
lon_fix_1 = df.where(df.lon < 540).lon - 360
lon_fix_2 = df.where(df.lon >= 540).lon - 720
lon_fix = lon_fix_1
lon_fix = lon_fix_1.fillna(lon_fix_2)
df.lon = lon_fix

In [ ]:
df.head()

__radius__

In [ ]:
radius = df.L*1000
df.L = radius
df = df.rename(columns = {'L': 'radius'})

In [ ]:
df.head()

### mongodb ###

In [ ]:
from pymongo import MongoClient
from tqdm import tqdm

In [ ]:
track = np.asarray(df.track)
n = np.asarray(df.n)
date = df.date
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)
radius = np.asarray(df.radius)

In [ ]:
tra_num = max(track)
tra_num

In [ ]:
obs_num = len(n)
obs_num

In [ ]:
df.head()

In [ ]:
# collection
ssh_cchdo = MongoClient().eddies.ssh_cchdo

In [ ]:
for i in tqdm(range(obs_num)):
    eddy = {
        'eid': str(track[i]) + "-" + str(n[i]),
        'dat': date[i],
        'loc': {
            'type': 'Point',
            'coordinates': [float(lon[i]), float(lat[i])]
        },
        'rad': float(radius[i])
    }
    ssh_cchdo.insert_one(dict(eddy))